In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from textblob import Word, TextBlob


In [ ]:
df=pd.read_csv("/kaggle/input/phishing-email-dataset/CEAS_08.csv")

In [ ]:
#Phishing (1) vs. Legitimate (0)

In [ ]:
df.head()

In [ ]:
df["sender"].value_counts()

In [ ]:
df["sender_2"]=df['sender'].str.split('@').str[1]

In [ ]:
df["sender_2"].value_counts()

In [ ]:
nltk.download('vader_lexicon')

In [ ]:
sia=SentimentIntensityAnalyzer()

In [ ]:
sia.polarity_scores("It is terrible")["compound"]

In [ ]:
def get_compound_score(text):
    return sia.polarity_scores(text)['compound']

In [ ]:
def get_compound_score(text):
    return sia.polarity_scores(text)['compound']

df['subject'] = df['subject'].astype(str)
df['body'] = df['body'].astype(str)


In [ ]:
df['Compound_Subject'] = df['subject'].apply(get_compound_score)


In [ ]:
df['Compound_Body'] = df['body'].apply(get_compound_score)

In [ ]:
df["Compound"]=df['Compound_Subject'].astype(float)+df['Compound_Body'].astype(float)

In [ ]:
bins = [-2, -1, 0, 1,2]
labels = ['-2 to -1', '-1 to 0', '0 to 1', '1 to 2']

# Yeni etiketlenmiş sütunu oluşturun
df['Compound_Label'] = pd.cut(df['Compound'], bins=bins, labels=labels, include_lowest=True)

In [ ]:
df.head()

In [ ]:
df["subject"]=df["subject"].str.lower()

In [ ]:
df["body"]=df["body"].str.lower()

In [ ]:
df["Subject+Body"]=df["subject"]+" "+df["body"]

In [ ]:
df["Subject+Body"]=df["Subject+Body"].str.replace('[^\w\s]','')

In [ ]:
spam_words=pd.read_excel("/kaggle/input/spam-words/Spam_Words.xlsx")

In [ ]:
spam_words["SPAM WORDS"]=spam_words["SPAM WORDS"].str.lower()

In [ ]:
spam_words["SPAM WORDS"]=spam_words["SPAM WORDS"].str.replace('[^\w\s]','')

In [ ]:
def count_spam_words(text, spam_words):
    text = text.lower()  
    count = 0
    for word in spam_words["SPAM WORDS"]:
        if word in text:
            count += 1
    return count

In [ ]:
df["Spam_Count"]=df["Subject+Body"].apply(lambda x: count_spam_words(x, spam_words))

In [ ]:
df["Text_Length"] = df["Subject+Body"].apply(lambda x: len(x))

In [ ]:
df['edu_mail'] = df['sender'].apply(lambda x: 1 if 'edu' in x else 0)
df['python_mail'] = df['sender'].apply(lambda x: 1 if 'python' in x else 0)
df['apache_mail'] = df['sender'].apply(lambda x: 1 if 'apache' in x else 0)
df['loewis_mail'] = df['sender'].apply(lambda x: 1 if 'loewis' in x else 0)
df['gmail_mail'] = df['sender'].apply(lambda x: 1 if 'gmail' in x else 0)
df['org_mail'] = df['sender'].apply(lambda x: 1 if 'org' in x else 0)


In [ ]:
df.head()

In [ ]:
df_new=df[["urls","Compound_Subject","Spam_Count","Text_Length","edu_mail","python_mail","apache_mail","loewis_mail","gmail_mail","org_mail","label"]]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

correlation_matrix = df_new.corr()

plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Korelasyon Matrisi Isı Haritası')
plt.show()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
TFIDV=TfidfVectorizer()



In [ ]:
from scipy.sparse import hstack, csr_matrix

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_features=10000, max_df=0.95, min_df=2)
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Subject+Body'])

tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

sparse_features = csr_matrix(df[["urls", "Compound_Subject", "Spam_Count", "Text_Length", "edu_mail", "python_mail", "apache_mail", "loewis_mail", "gmail_mail", "org_mail"]].values)

X = hstack([sparse_features, tfidf_matrix])

In [ ]:
y=df["label"]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Model Accuracy:", accuracy)

In [ ]:
X_2=tfidf_df


In [ ]:
X_2_train, X_2_test, y_train, y_test = train_test_split(X_2, y, test_size=0.3, random_state=42)


In [ ]:
model = LogisticRegression()
model.fit(X_2_train, y_train)

y_pred = model.predict(X_2_test)

accuracy = accuracy_score(y_test, y_pred)
print("Model Accuracy:", accuracy)

In [ ]:
X_3=df[["urls", "Compound_Subject", "Spam_Count", "Text_Length", "edu_mail", "python_mail", "apache_mail", "loewis_mail", "gmail_mail", "org_mail"]]

In [ ]:
X_3_train, X_3_test, y_train, y_test = train_test_split(X_3, y, test_size=0.3, random_state=42)

In [ ]:
#BEST OPTION IS X_2

In [ ]:
model = LogisticRegression()
model.fit(X_3_train, y_train)

# Test verileri ile tahmin yapma
y_pred = model.predict(X_3_test)

# Doğruluk oranını hesaplama
accuracy = accuracy_score(y_test, y_pred)
print("Model Accuracy:", accuracy)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model = RandomForestClassifier()
model.fit(X_2_train, y_train)

y_pred = model.predict(X_2_test)

accuracy = accuracy_score(y_test, y_pred)
print("Model Accuracy:", accuracy)